In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/refine-epitope-deep-learning')

Mounted at /content/drive/


In [2]:
!pip install transformers
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.6 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 6.6 MB 66.7 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 9.0 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 209 kB 56.4 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 112 kB 67.2 MB/s 
     |████████████████████████████████| 146 kB 75.7 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
  Created wheel for pyp

# Preprocess data 

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

In [4]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [5]:
def preprocess_data(data):

    # Preprocess data
    X = list(data["sequence"])
    y = list(data["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)
    return train_dataset, val_dataset

In [6]:
df_train = pd.read_csv("./input/data_train.csv")

sequence_formatted = []
for seq in df_train['sequence'].values:
  sequence_formatted.append(" ".join(seq))

data = pd.DataFrame({'sequence':sequence_formatted, 'label':df_train['label'].tolist()})

#data = df_train

data_op = data[:int(len(data)/2)]


# Define pretrained tokenizer and model
batch_size=8
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_dataset_op, val_dataset_op = preprocess_data(data_op)
train_dataset, val_dataset = preprocess_data(data)

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    f"{model_name}-finetuned-classification",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    optim="adamw_torch"
)

def model_init():
    return model

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset_op,
    eval_dataset=val_dataset_op,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
[I 2022-07-06 21:54:08,532] Trial 7 pruned. 
Trial:
***** Running training *****
  Num examples = 330
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 126


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.448385,0.746988,0.523810,0.956522,0.676923


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
[I 2022-07-06 21:54:25,724] Trial 8 pruned. 
Trial:
***** Running training *****
  Num examples = 330
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 84


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.387240,0.831325,0.680000,0.739130,0.708333


***** Running Evaluation *****
  Num examples = 83
  Batch size = 8
[I 2022-07-06 21:54:42,918] Trial 9 pruned. 


## Set the model with the best parameters and run it on the full dataset

In [7]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train_dataset=train_dataset
trainer.eval_dataset=val_dataset

trainer.train()

***** Running training *****
  Num examples = 660
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 166


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.283431,0.867470,0.854167,0.732143,0.788462
2,No log,0.272089,0.879518,0.860000,0.767857,0.811321


***** Running Evaluation *****
  Num examples = 166
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/checkpoint-83
Configuration saved in bert-base-uncased-finetuned-classification/checkpoint-83/config.json
Model weights saved in bert-base-uncased-finetuned-classification/checkpoint-83/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/checkpoint-83/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-classification/checkpoint-83/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 166
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-classification/checkpoint-166
Configuration saved in bert-base-uncased-finetuned-classification/checkpoint-166/config.json
Model weights saved in bert-base-uncased-finetuned-classification/checkpoint-166/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-classification/checkpoint-166/to

TrainOutput(global_step=166, training_loss=0.32443432635571584, metrics={'train_runtime': 91.9071, 'train_samples_per_second': 14.362, 'train_steps_per_second': 1.806, 'total_flos': 655948194343200.0, 'train_loss': 0.32443432635571584, 'epoch': 2.0})

In [8]:
# ----- 3. Predict -----#
# Load test data
#test_data = pd.read_csv("test.csv")
test = pd.read_csv("./input/data_test.csv")

sequence_formatted = []
for seq in test['sequence'].values:
  sequence_formatted.append(" ".join(seq))

test_data = pd.DataFrame({'sequence':sequence_formatted, 'label':test['label'].tolist()})


X_test = list(test_data["sequence"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)


***** Running Prediction *****
  Num examples = 207
  Batch size = 8


In [9]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("ROC_AUC:", roc_auc_score(test_data['label'], y_pred))

print(classification_report(test_data['label'], y_pred))

ROC_AUC: 0.8216195569136746
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       119
           1       0.86      0.73      0.79        88

    accuracy                           0.84       207
   macro avg       0.84      0.82      0.83       207
weighted avg       0.84      0.84      0.83       207

